In [16]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
import nltk
nltk.download('punkt_tab')




[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [21]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer


In [11]:
df = pd.read_csv("/content/drive/Othercomputers/Mon ordinateur portable/Data Science/Spinning/Base.csv")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58424 entries, 0 to 58423
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  58424 non-null  int64 
 1   indi        58424 non-null  int64 
 2   cuerpo      58424 non-null  object
 3   titular     58424 non-null  object
dtypes: int64(2), object(2)
memory usage: 1.8+ MB


In [13]:
texts = df["cuerpo"]
texts

,cuerpo
0,dos semanas después de su puesta de largo y pr...
1,"este viernes, el presidente del gobierno, pedr..."
2,"el ministro del interior, fernando grande-marl..."
3,son días muy duros para la familia de olivia y...
4,sólo quedan 10 presos de eta por recibir los b...
...,...
58419,la comisión europea inició este un procedimien...
58420,el pleno de la asamblea de madrid ha aprobado ...
58421,la comisión de investigación parlamentaria del...
58422,"erc y pdecat han calificado este jueves de ""in..."


In [6]:
texts[0]

'dos semanas después de su puesta de largo y presentación en sociedad, el primer submarino s-80 para la armada, el s-81 "isaac peral", ha entrado hoy en el agua tras una delicada y larga maniobra que se ha retrasado varios días por las condiciones meteorológicas. de esta forma, tras completar su construcción 17 años después de que arrancara el programa, navantia ha cumplido otro importante hito.españa.submarino s-80 tras 17 años y 3.900 millones, el "isaac peral" ya está aquíespaña.el comandante del s-81 isaac peral» va a ser un revulsivo para la armada»desde que el pasado día 22 la princesa leonor amadrinara el que es considerado uno de los submarinos (no nucleares) más avanzados del mundo, en el astillero de navantia de cartagena (murcia) comenzaron las tareas que llevarían al s-80 al agua. un lento y delicado trabajo que comenzó con su colocación sobre un dique flotante. tras esto, se ha hecho flotar el dique y se ha trasladado hasta la fosa, el lugar de la dársena que cuenta con el

In [14]:
probas = {} # key: (w(t-1), w(t+1)) , value: {w(t): count(w(t))} # this is what we want to obtain

In [22]:
# lets split by sentence.
# tokens_1 produit des unités linguistiques plus précises et mieux adaptées
# pour une analyse linguistique fine, tandis que tokens_2 donne simplement des
# sous-chaînes séparées par des espaces

for text in tqdm(texts) :
  sentences = text.split(".")
  for sentence in sentences :
    tokens = word_tokenize(sentence, language = 'spanish') #tokens_2 = sentence.split() moins bien
    if (len(tokens) >=2):
      for i in range(len(tokens)-2): # to keep in the range
        #probs[(tokens[i],tokens(i+2))] ={token[i+1] : }
        t0 = tokens[i] #for key
        t1 = tokens[i+1] #value
        t2 = tokens[i + 2] #for key
        key = (t0,t2)
        if key not in probas: #if dict never saw this couple
          probas[key] = {}
        if t1 not in probas[key]: #if the dict probas[key] doesnt countain t1
          probas[key][t1] = 1 #initialize
        else:
          probas[key][t1] += 1 #increment




100%|██████████| 58424/58424 [08:02<00:00, 121.08it/s]


In [23]:
probas

Output hidden; open in https://colab.research.google.com to view.

In [24]:
# divide to get probabilities
for  key, v in (probas.items()):
  tot_count=0
#  print(f"key: {key}, value: {v}")
  tot_count = sum(v.values())
#  print(f"totcount {tot_count}")
  for subkey, subvalue in (v.items()):
    probas[key][subkey] /= tot_count #probas[key][subkey] == v[subkey]




In [ ]:
probas

In [25]:
untokenize = TreebankWordDetokenizer()
print(untokenize.detokenize(["CC","jsuis","ici"]))

CC jsuis ici


In [26]:
def sample_word(dico): #we already saw this method in the previous notebook, with text generation
  p0 = np.random.random()
  cumul = 0
  for k, v in dico.items():
    cumul += v
    if cumul >= p0:
      return k

#Choix basé sur le seuil cumulatif :

# La fonction commence à additionner les valeurs (probabilités) et s’arrête
# dès que la somme cumulée dépasse ou atteint le seuil aléatoire p0. Ainsi,
# même si les probabilités ne sont pas triées, chaque mot a la probabilité
# de sortir qui correspond à sa valeur. Principe de la sélection par somme
# cumulative :

# En prenant une valeur aléatoire entre 0 et 1 et en cumulant les probabilités
# jusqu'à dépasser cette valeur, la sélection respecte les poids donnés dans
# le dictionnaire. Par exemple, si une probabilité est plus élevée, sa
# contribution au cumul sera plus grande, et elle sera donc plus susceptible
# d’être atteinte rapidement, indépendamment de sa position dans dico. %%


In [31]:
sample_word(probas[('la', ',')])

'pnl'

In [32]:
def spin_line(line):
  tokens = word_tokenize(line, language='spanish')
  i = 0
  output = [tokens[0]] #first word
  if len(tokens) >= 2:
    while i < (len(tokens) - 2):
      t0 = tokens[i] #word before
      t1 = tokens[i+1] #word we want to spin
      t2 = tokens[i+2]#word after
      key = (t0,t2)
      prob_dico = probas[key]
      if len(prob_dico) > 1 and np.random.random() < 0.3:
        middle = sample_word(prob_dico)
        output.append(t1)
        output.append("<" + middle + ">")
        output.append(t2)
        i =+ 2
      else:
        output.append(t1)#only get the next word
        i += 1
    if i == (len(tokens) -2):
      output.append(tokens[-1]) #last word of the token
    return untokenize.detokenize(output)




In [34]:
spin_line("dos semanas después de su puesta")

'dos semanas después <advirtiendo> de de su <la> puesta de <de> su de su <la> puesta de su <la> puesta de <de> su de su puesta'

In [35]:

def spin_document(doc):
    lineas = doc.split(".")
    output = []
    for linea in lineas:
        if linea:
            new_line = spin_line(linea)
        else:
            new_line = linea
        output.append(new_line)
    return "\n".join(output)



In [48]:

doc = 'el concejal de compromís en el ayuntamiento de burjassot (valència) ángel vázquez sufrió anoche una agresión homófoba en valència'


In [50]:
new_doc = spin_document(doc)

In [51]:
new_doc

'el concejal de compromís <horario> en compromís <acs> en compromís en el ayuntamiento <desarrollo> de compromís en el <el> ayuntamiento compromís en el ayuntamiento de burjassot (valència) ángel vázquez sufrió anoche una agresión <agresión> homófoba compromís <españa> en compromís <trabajar> en compromís en el ayuntamiento <respeto> de compromís <base> en compromís <villarejo> en compromís <ciudadanos> en compromís <conflictividad> en compromís en el ayuntamiento de burjassot (valència) ángel <luis> vázquez compromís en el <el> ayuntamiento compromís en el <el> ayuntamiento compromís <prensa> en compromís en el ayuntamiento de burjassot (valència) ángel vázquez sufrió anoche una agresión homófoba <sexual> en compromís en el <el> ayuntamiento compromís en el <el> ayuntamiento compromís en el ayuntamiento de burjassot <compostela> (compromís en el <el> ayuntamiento compromís <podemos> en compromís en el <el> ayuntamiento compromís en el ayuntamiento de burjassot <baltimore> (compromís e